Kacie's Scripts

In [ ]:
year = None
month = None
program = "prepilot"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install seaborn
import seaborn as sns
from collections import defaultdict
import datetime

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
import scaffolding 
from plots import *

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_ei
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
tq = scaffolding.get_time_query(year, month)

In [ ]:
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)

In [ ]:
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)

In [ ]:
expanded_ct = scaffolding.expand_userinputs(labeled_ct)

In [ ]:
expanded_ct.shape

In [ ]:
expanded_ct = scaffolding.data_quality_check(expanded_ct)
expanded_ct.shape

In [ ]:
## Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

In [ ]:
dic_pur

In [ ]:
assert len(expanded_ct[(expanded_ct['Mode_confirm'] == 'Pilot ebike') & (expanded_ct["Replaced_mode"] == "Pilot ebike")]) == 0

In [ ]:
# Energy Impact Calculation
scaffolding.unit_conversions(expanded_ct)

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

Demographics

In [ ]:
#All Project Participants - includes stage users and participant users
#Participant users - excludes stage users

In [ ]:
#All the demographic survey entries
demog = pd.read_csv("Can Do Colorado eBike Program - en.csv")
#demog.columns
demog.columns = ['Timestamp','user_id','YearOfBirth','Gender','DriversLicense','Student','HighestDegree','Work','MoreThanOneJob','FullPartTime','Job','TranspToJobLastWeek','DailyCommuteMinsToJob','ChangeStartTime','CanWFH','DaysPerWeekWFT','DescribesYouBest','OwnRentResidence','HomeType','Income','PplInHome','ChildrenUnder18','NumOfDrivers','NumVehicles','AltTranspt','MedCond','HowLongMedCond']
demog

es_users = pd.read_csv("Can Do Colorado eBike Program - es.csv")

# this translates the spanish df to the english df

es_users.columns = demog.columns # translate cols to english
pd.set_option('display.max_columns', None)
# the rest of this is a little clunky, but it's good for the plots I make (so far)
es_users = es_users.replace("Menos de $24,999", "Less than $24,999")
es_users = es_users.replace("Femenino.", "Woman")
es_users = es_users.replace("Masculino.", "Man")
es_users = es_users.replace("Si", "Yes")
es_users = es_users.replace("Bicicleta", "Bicycle")
es_users = es_users.replace("Teletrabajo", "Telecommute")
es_users = es_users.replace("Caminar", "Walk")
es_users = es_users.replace("Graduado de secundaria o GED.", "High school graduate or GED")
en_es_users = demog.append(es_users) #df of both english and spanish
en_es_users

In [ ]:
#df of all project partcipants, including only the last survey entry, no duplicates
#First survey entry 5-23-2021
#Last survey entry on 5-10-2022
pd.set_option('display.max_rows', 10)
demog = en_es_users
demog1 = demog.sort_values(by = 'Timestamp')
demogRecent = demog1.drop_duplicates(subset='user_id',keep = 'last')
demogRecent=demogRecent.reset_index()
demogRecent

In [ ]:
#Dataframe that includes all survey entries that are more than 2 weeks apart
demogSep = demog.drop([247,123,254,262,30,256,33,55,70,114,239,251,237,238,206,21,219,23,94,95,121,106,91,63,59,280,283,13,182,56,233,90,60,103,153,176,145,135,27,101,132,217,5,6,1,29])
#demogSep.user_id.value_counts()

In [ ]:
#Less than $24,999
#$25,000-$49,999
#$50,000-$99,999
#$100,000 -$149,999
#$150,000-$199,999
#$200,000 or more
#Prefer not to say
def returnIncomeDistribution(demog,title):
    lessThan24=0
    i25to49=0
    i50to99=0
    i100to149=0
    i150to199=0
    i200orMore=0
    NA=0
    
    for i in range(len(demog)):
        if str(demog.Income.iloc[i]).__contains__('24,999'):
            lessThan24+=1
        elif str(demog.Income.iloc[i]).__contains__('25,000'):
            i25to49+=1
        elif str(demog.Income.iloc[i]).__contains__('100,000'):
            i100to149+=1
        elif str(demog.Income.iloc[i]).__contains__('150,000'):
            i150to199+=1
        elif str(demog.Income.iloc[i]).__contains__('50,000'):
            i50to99+=1
        elif str(demog.Income.iloc[i]).__contains__('200,000'):
            i200orMore+=1
        elif str(demog.Income.iloc[i]).__contains__('Prefer'):
            NA+=1    

    values = [lessThan24,i25to49,i50to99,i100to149,i150to199,i200orMore,NA]
    income = ['Less than $24,999','\$25,000-\$49,999','\$50,000-\$99,999','\$100,000-\$149,999','\$150,000-\$199,999','$200,000 or more','Prefer not to say']
    colours = dict(zip(values, plt.cm.tab20.colors[:len(values)]))
    def func(pct, values):
            total = sum(values)
            absolute = int(round(pct*total/100.0))
            return "{:.1f}%\n({:d})".format(pct, absolute) if pct > 3 else''
    fig = plt.figure(figsize =(10, 10))
    plt.pie(values, labels = income, colors=[colours[key] for key in values], pctdistance=.75, autopct=lambda pct: func(pct, values),textprops={'size': 16})
    plt.title(title, fontsize=25)
    plt.show()

In [ ]:
def AgeDistribution(demog,title):
    demog['Age']=0
    for i in range(len(demog)):
        #print('hi',i)
        if int(demog.iloc[i].YearOfBirth) > 22 and int(demog.iloc[i].YearOfBirth) < 1000:
            demog.at[i,'Age']=2022-(int(demog.iloc[i].YearOfBirth)+1900)
        else:
            #print('false')
            demog.at[i,'Age']=2022-int(demog.YearOfBirth.iloc[i])
    age = [demog.Age]
    plt.hist(age, bins = len(demog.Age.unique()))
    plt.title(title)
    plt.xlabel('Age')
    plt.ylabel('Number of Participants')
    plt.show()

In [ ]:
#def AgeDistribution(demog):
#    demog=demog.assign(Age=2022-demogRecent.YearOfBirth)
#    demog1=demog.reset_index()
#    for i in range(len(demog1)):
#        if demog1.loc[i].YearOfBirth > 22 and demog1.loc[i].YearOfBirth < 1000:
#            demog1.at[i,'Age']=2022-(demog1.loc[i].YearOfBirth+1900)
#    age = [demog1.Age]
#    plt.hist(age, bins = 13)
#    plt.title('Age Distribution of Participants')
#    plt.xlabel('Age')
#    plt.ylabel('Number of Participants')
#    plt.show()

In [ ]:
#under development - trying to combine (Car, SUV, Van) and (Public or commuter bus, Buss)
#also want to make Motor Vehicle blue and not gray, at the top of list
def MainTransport(mainmode,title):
    labels = mainmode.value_counts().index # the labels in the pie chart legend
    
    sizes = mainmode.value_counts()/len(mainmode)*100
    vehNum = np.array([0],dtype=np.float_)
    for i in range(len(labels)):
        if labels[i] == 'Car' or labels[i] == 'SUV' or labels[i] == 'Van':
            vehNum+=sizes[i]
        elif labels[i] == "Buss":
            sizes['Public or commuter bus']+=sizes[i]
    veh= pd.Index(['Motor Vehicle (Car, SUV, Van)'])
    labels = labels.append(veh)
    labels=labels.drop('Buss')
    labels=labels.drop('Car')
    labels=labels.drop('Van')
    labels=labels.drop('SUV')
    sizes=sizes.drop('Buss')
    sizes=sizes.drop('Car')
    sizes=sizes.drop('Van')
    sizes=sizes.drop('SUV')
    sizes['Motor Vehicle (Car, SUV, Van)']=vehNum[0]
    #ran = int(len(labels))
    #for i in range(ran):
        #if labels[i] == 'Car' or labels[i] == 'SUV' or labels[i] == 'Van':
        #    labels=labels.append(veh)
            #labels = labels[~labels.index.duplicated(keep='first')]
        #elif labels[i] == "Buss":
        #    labels=labels.drop('Buss')
            #labels = labels.value_counts().index
    #sizes=sizes.reset_index()
    #labels=labels.reset_index()
    #sizes[sizes[veh]]=vehNum
    colours = dict(zip(sizes, plt.cm.tab20.colors[:len(sizes)]))
    def func(pct, sizes):
            total = sum(sizes)
            absolute = int(round(pct*total/100.0))
            return "{:.1f}%\n({:d})".format(pct, absolute) if pct > 4 else''

    fig = plt.figure(figsize =(10, 10))
    plt.pie(sizes, labels = labels, colors=[colours[key] for key in sizes], pctdistance=.75, autopct=lambda pct: func(pct, sizes),textprops={'size': 16})
    plt.title(title, fontsize=25)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = labels)
    plt.show()
    return labels

MainTransport(demogRecent.TranspToJobLastWeek,'Main Mode of Transportation for All Participants')


In [ ]:
def genderDistrb(Gender,title):
    counts = Gender.value_counts()
    labels = Gender.value_counts().index # the labels in the pie chart legend
    sizes = Gender.value_counts()/len(Gender)*100 # the percentages to display
    labelz = [f'{l}, {s:0.1f}%' for l, s in zip(labels, sizes)]
    colours = dict(zip(counts, plt.cm.tab20.colors[:len(counts)]))
    def func(pct, counts):
            total = sum(counts)
            absolute = int(round(pct*total/100.0))
            return "{:.1f}%\n({:d})".format(pct, absolute) if pct > 4 else''
    fig = plt.figure(figsize =(10, 10))
    plt.pie(counts, labels = labels, colors=[colours[key] for key in counts], pctdistance=.75, autopct=lambda pct: func(pct, counts),textprops={'size': 16})
    plt.title(title, fontsize=25)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = labelz)
    plt.show()

In [ ]:
def MainTransport(mainmode,title):
    mainmode.value_counts().plot(title = title, kind = "pie", figsize = (5,5),
                            labels = None, )

    labels = mainmode.value_counts().index # the labels in the pie chart legend
    sizes = mainmode.value_counts()/len(mainmode)*100 # the percentages to display
    labels = [f'{l}, {s:0.1f}%' for l, s in zip(labels, sizes)]
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = labels)
    plt.show()

In [ ]:
#demo is the demographic df
def haveCar(demo):
    no=0
    yes=0
    demo['HasCar']=""
    for i in range(len(demo)):
        if len(str(demo.iloc[i].NumVehicles)) == 1:
            if int(demo.iloc[i].NumVehicles) == 0:
                demo.at[i,'HasCar'] = "No"
                no+=1
            elif int(demo.iloc[i].NumVehicles) != 0:
                demo.at[i,'HasCar'] = "Yes"
                yes+=1
        elif len(str(demo.iloc[i].NumVehicles)) == 2:
            demo.at[i,'HasCar'] = "Yes"
            yes+=1
    values=[yes,no]
    have=["Has a car","Doesn't have a car"]
    colours = dict(zip(values, plt.cm.tab20.colors[:len(values)]))
    def func(pct, values):
            total = sum(values)
            absolute = int(round(pct*total/100.0))
            return "{:.1f}%\n({:d})".format(pct, absolute) if pct > 4 else''
    fig = plt.figure(figsize =(10, 10))
    plt.pie(values, labels = have, colors=[colours[key] for key in values], pctdistance=.75, autopct=lambda pct: func(pct, values),textprops={'size': 16})
    plt.title('Distribution Of Car Owners', fontsize=25)
    plt.show()

In [ ]:
#Demographics of all project participants
returnIncomeDistribution(demogRecent,'Income Distribution of All Project Participants')
AgeDistribution(demogRecent,'Age Distribution of All Project Participants')
genderDistrb(demogRecent.Gender,'Gender Distribution of All Participants')
MainTransport(demogRecent.TranspToJobLastWeek,'Main Mode of Transportation for All Participants')
haveCar(demogRecent)

In [ ]:
#maps user_email to user_id
import emission.core.get_database as edb 

#list(edb.get_uuid_db().find({}, {"user_email":1, "_id":0}))
list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id":0}))
token = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id":0})))
token = token.rename(columns={"uuid":"user_id"})
for i in range(len(token)):
    token.user_id.iloc[i] = str(token.user_id.iloc[i]).replace("-","")

token

In [ ]:
df = demogRecent.merge(token,left_on=['user_id'],right_on=['user_id'],how='left')
df = df[df['user_email'].notnull()]
dfPar = df[df.user_email.str.startswith(('vail_','cc_','cc_ ','sc_','fc_','pc_','4c_'))]
dfPar = dfPar.reset_index()

participantDemog = dfPar
#participantDemog

In [ ]:
#Participant user demographics       
df = demogRecent.merge(token,left_on=['user_id'],right_on=['user_id'],how='left')
df = df[df['user_email'].notnull()]
dfPar = df[df.user_email.str.startswith(('vail_','cc_','cc_ ','sc_','fc_','pc_','4c_'))]
dfPar = dfPar.reset_index()

participantDemog = dfPar
participantDemog
returnIncomeDistribution(participantDemog,'Income Distribution of Participant Users')
AgeDistribution(participantDemog,'Age Distribution of Participant Users')
genderDistrb(participantDemog.Gender,'Gender Distribution of Participant Users')
MainTransport(participantDemog.TranspToJobLastWeek,'Main Mode of Transportation for Participant Users')
haveCar(participantDemog)

In [ ]:
#Stage user demographics
df = demogRecent.merge(token,left_on=['user_id'],right_on=['user_id'],how='left')
df = df[df['user_email'].notnull()]
dfStage = df[df.user_email.str.startswith('stage')]
dfStage = dfStage.reset_index()
stageDemog = dfStage

returnIncomeDistribution(stageDemog,'Income Distribution of Stage Users')
AgeDistribution(stageDemog,'Age Distribution of Stage Users')
genderDistrb(stageDemog.Gender,'Gender Distribution of Stage Users')
MainTransport(stageDemog.TranspToJobLastWeek,'Main Mode of Tranportation for Stage Users')
haveCar(stageDemog)

In [ ]:
#tells me if a uuid exists in a df (for testing purposes)
def exists(user_id, df):
    for i in range(len(df)):
        if df.iloc[i].user_id == user_id:
            print("True")

In [ ]:
#NOT BEING USED
#Max's script
# takes list of user id's and returns the overlap in en_usr and es_usr
def usr_overlap(user_id):
    df_mode_users = demogRecent[demogRecent.iloc[:,1].isin(user_id)]
    return df_mode_users

# create df of all users from a certain program
def df_from_program(program):
    df = all_users.loc[all_users['user_email'].str.contains(program)]
    prog_users = df['uuid'].astype(str)
    prog_users = [i.replace('-','') for i in prog_users] # remove all dashes from strings
    return usr_overlap(prog_users)

all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
#stageDemog = df_from_program('stage_')

# create a df from all users except from the stage program
all_str_but_stage = 'cc_|sc_|vail_|fc_|pc_|4c_'
all_usr_but_stage = all_users.loc[all_users['user_email'].str.contains(all_str_but_stage)]

prog_users = all_usr_but_stage['uuid'].astype(str)
prog_users = [i.replace('-','') for i in prog_users] # remove all dashes from strings
#participantDemog = usr_overlap(prog_users)

def age_plots(df, title):
    ages = 2022- df['YearOfBirth']
    ages = ages[0 < ages < 140] # todo: maybe revisit this way of catching erroneous ages
    plt.hist(ages, bins = len(ages.unique()))
    plt.title(title)
    plt.ylabel('Frequency')
    plt.xlabel('Age (years)')
    plt.xlim()
    plt.show()

#age_plots(demogRecent, 'All Users Demographic Age Distribution')

#age_plots(stageDemog, 'Stage Demographic Age Distribution')

#age_plots(participantDemog, 'Participant Demographic Age Distribution')

E-bike usage data

In [ ]:
#RETURN DF OF ONLY PARTICIPANT USERS IN EXPANDED_CT
for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
df = expanded_ct.merge(token,left_on=['user_id'],right_on=['user_id'],how='left')
df = df[df['user_email'].notnull()]
dfPar = df[df.user_email.str.startswith(('vail_','cc_','cc_ ','sc_','fc_','pc_','4c_'))]
dfPar = dfPar.reset_index()

participant = dfPar

In [ ]:
#returns every users proportion of ebike trips by month with proportion less than .1

def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2022']=percent
    #keeps rows whose max - min < .1
    for i in timeline.index:
        if timeline.loc[i].max() - timeline.loc[i].min() > .1:
            timeline.drop([i],inplace=True)
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.show

get_ebike_percent_byMonth(participant)

In [ ]:
#returns every users proportion of ebike trips by month
#difference b/w max and min > .1, with max in 1st half, but not good visualization bc still hard to see
df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
import statistics
def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2022']=percent
    #keeps rows whose max - min < .1
    #return timeline.index
    #return timeline.loc['87cb88529b614c0e91ec07a33145659b'][0]==None
    #return statistics.mean(timeline.iloc[0]!=None)
    for i in timeline.index:
        #return timeline.loc[i][:int(len(timeline.loc[i])/2)]==0
        max = timeline.loc[i].max()
        #return statistics.mean(timeline.loc[i])
        if timeline.loc[i].max() - timeline.loc[i].min() < .1:
            timeline.drop([i],inplace=True)
        else:
            k = int(len(timeline.loc[i])/2)
            #print(i,':',k)
            #print(type(timeline.loc[i][k]))
            first=[]
            for j in range(k):
                #return timeline.loc[i][0]==None
                #return timeline.loc[i][j]==None
                if timeline.loc[i][j] == None:
                    if int(timeline.loc[i][j]) != max:
                        timeline.drop([i],inplace=True)
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.iloc[-1],y=timeline.iloc[i],data=timeline,marker='o')
        plt.show
        
    #for i in range(len(timeline1)-1):
     #   sns.lineplot(ax=ax,x=timeline1.loc[timeline1.index[-1]],y=timeline1.iloc[i],data=timeline,marker='o')
     #   plt.show

get_ebike_percent_byMonth(participant)

In [ ]:
#returns every users proportion of ebike trips by month
#ALL PROJECT PARTCIPANTS
def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2022']=percent
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    #return user['month'+str(7)][2]
    #pd.set_option('display.max_rows', None)
    #return timeline
    #return timeline.iloc[1,0]
    #return countOfUsers
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.show

get_ebike_percent_byMonth(df)

In [ ]:
#returns every users proportion of ebike trips by month
#PARTICIPANT USERS ONLY

for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
#expanded_ct

df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
#19-24,25-34,35-44,45-54,55+
df['Age_group']=''
for i in range(len(df)):
    if df.Age.iloc[i] >= 19 and df.Age.iloc[i] <= 24:
        df.at[i,'Age_group'] = '19-24'
    if df.Age.iloc[i] >= 25 and df.Age.iloc[i] <= 34:
        df.at[i,'Age_group'] = '25-34'
    if df.Age.iloc[i] >= 35 and df.Age.iloc[i] <= 44:
        df.at[i,'Age_group'] = '35-44'
    if df.Age.iloc[i] >= 45 and df.Age.iloc[i] <= 54:
        df.at[i,'Age_group'] = '45-54'
    if df.Age.iloc[i] >= 55:
        df.at[i,'Age_group'] = '55+'
df = df[df['Timestamp'].notnull()]

def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].user_id.iloc[0]),'May 2022']=percent
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    #timeline["Age_group"]=demo.user_id.Age_group
    #return user['month'+str(7)][2]
    #pd.set_option('display.max_rows', None)
    #return timeline.iloc[0]
    #return timeline.iloc[1,0]
    #return countOfUsers
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.show

get_ebike_percent_byMonth(participant)

In [ ]:
#returns the AVERAGE of proportion of ebike trips by month for all users

for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
#expanded_ct

df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
#19-24,25-34,35-44,45-54,55+
df['Age_group']=''
for i in range(len(df)):
    if df.Age.iloc[i] >= 19 and df.Age.iloc[i] <= 24:
        df.at[i,'Age_group'] = '19-24'
    if df.Age.iloc[i] >= 25 and df.Age.iloc[i] <= 34:
        df.at[i,'Age_group'] = '25-34'
    if df.Age.iloc[i] >= 35 and df.Age.iloc[i] <= 44:
        df.at[i,'Age_group'] = '35-44'
    if df.Age.iloc[i] >= 45 and df.Age.iloc[i] <= 54:
        df.at[i,'Age_group'] = '45-54'
    if df.Age.iloc[i] >= 55:
        df.at[i,'Age_group'] = '55+'
demo = df[df['Timestamp'].notnull()]

def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'user_id':[],'Date':[],'percent':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            timeline=timeline.append({'user_id': user['month'+str(i)][j].user_id.iloc[0],'Date': pd.to_datetime(str(user['month'+str(i)][j].iloc[0].start_local_dt_year)+'-'+str(user['month'+str(i)][j].iloc[0].start_local_dt_month)),'percent':percent}, ignore_index=True)
                      #      
    #return timeline
    #return user['month'+str(7)][2]
    #pd.set_option('display.max_rows', None)
    #return timeline.loc[timeline.index[-1]]
    #return timeline[0:125].to_string(index=False,header=False)
    #return countOfUsers
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    #for i in range(len(timeline)-1):
    sns.lineplot(x=timeline.Date,y=timeline.percent,data=timeline,marker='o')
    plt.show

get_ebike_percent_byMonth(participant)

In [ ]:
#returns every users proportion of ebike trips by month by age group
for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
#expanded_ct

df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
#19-24,25-34,35-44,45-54,55+
df['Age_group']=''
for i in range(len(df)):
    if df.Age.iloc[i] >= 19 and df.Age.iloc[i] <= 24:
        df.at[i,'Age_group'] = '19-24'
    if df.Age.iloc[i] >= 25 and df.Age.iloc[i] <= 34:
        df.at[i,'Age_group'] = '25-34'
    if df.Age.iloc[i] >= 35 and df.Age.iloc[i] <= 44:
        df.at[i,'Age_group'] = '35-44'
    if df.Age.iloc[i] >= 45 and df.Age.iloc[i] <= 54:
        df.at[i,'Age_group'] = '45-54'
    if df.Age.iloc[i] >= 55:
        df.at[i,'Age_group'] = '55+'
demo = df[df['Timestamp'].notnull()]


def Monthly_ebike_propBy_AgeGroup(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfAgeGroups = trips.Age_group.value_counts().nunique()
    numOfGroups={}
    user={}
    for i in range(len(df)):
        numOfGroups['num'+str(i)]=df[i].Age_group.value_counts().nunique() #number of age groups in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('Age_group')] #f['u'+str(i)] is the month, the indexs are the age group      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfGroups['num'+str(i)]): #iterates through age groups per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Age_group.iloc[0]),'May 2022']=percent
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    #return user['month'+str(3)][0]
    #return timeline
    #return timeline.iloc[1,0]
    #timeline.sort_index()
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time by Age Group')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = timeline.index)
        plt.show

Monthly_ebike_propBy_AgeGroup(demo)

In [ ]:
#returns every users proportion of ebike trips by month with error bars

for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
#expanded_ct

df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
#19-24,25-34,35-44,45-54,55+
df['Age_group']=''
for i in range(len(df)):
    if df.Age.iloc[i] >= 19 and df.Age.iloc[i] <= 24:
        df.at[i,'Age_group'] = '19-24'
    if df.Age.iloc[i] >= 25 and df.Age.iloc[i] <= 34:
        df.at[i,'Age_group'] = '25-34'
    if df.Age.iloc[i] >= 35 and df.Age.iloc[i] <= 44:
        df.at[i,'Age_group'] = '35-44'
    if df.Age.iloc[i] >= 45 and df.Age.iloc[i] <= 54:
        df.at[i,'Age_group'] = '45-54'
    if df.Age.iloc[i] >= 55:
        df.at[i,'Age_group'] = '55+'
demo = df[df['Timestamp'].notnull()]

def get_ebike_percent_byMonth(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfUsers = trips.user_id.value_counts().nunique()
    numOfUsers={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfUsers['num'+str(i)]=df[i].user_id.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('user_id')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'user_id':[],'Date':[],'percent':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfUsers['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            timeline=timeline.append({'age_group': user['month'+str(i)][j].Age_group.iloc[0],'Date': pd.to_datetime(str(user['month'+str(i)][j].iloc[0].start_local_dt_year)+'-'+str(user['month'+str(i)][j].iloc[0].start_local_dt_month)),'percent':percent}, ignore_index=True)
                      #      
    #return timeline
    #return user['month'+str(7)][2]
    #pd.set_option('display.max_rows', None)
    #return timeline.loc[timeline.index[-1]]
    #return timeline[0:125].to_string(index=False,header=False)
    #return countOfUsers
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    #for i in range(len(timeline)-1):
    sns.lineplot(x=timeline.Date,y=timeline.percent,hue=timeline.age_group,err_style='bars',data=timeline,marker='o')
    plt.show

get_ebike_percent_byMonth(demo)

In [ ]:
#returns every users proportion of ebike trips by month by income
for i in range(len(expanded_ct)):
    expanded_ct.user_id.iloc[i] = str(expanded_ct.user_id.iloc[i]).replace("-","")
#expanded_ct
df = expanded_ct.merge(participantDemog,left_on=['user_id'],right_on=['user_id'],how='left')
df = df[df['Timestamp'].notnull()]

def Monthly_ebike_propBy_Income(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfIncome = trips.Income.value_counts().nunique()
    numOfGroups={}
    user={}
    for i in range(len(df)):
        numOfGroups['num'+str(i)]=df[i].Income.value_counts().nunique() #number of age groups in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('Income')] #f['u'+str(i)] is the month, the indexs are the age group      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfGroups['num'+str(i)]): #iterates through age groups per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            else:
                percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].Income.iloc[0]),'May 2022']=percent
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    #return user['month'+str(3)][0]
    #return timeline
    #return timeline.iloc[1,0]
    #timeline.sort_index()
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time by Income')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = timeline.index)
        plt.show

Monthly_ebike_propBy_Income(df)

In [ ]:
#Monthy ebike prop by income excluding higher income
demo1 = df
demo1=demo1.reset_index(drop=True)
demo1
#demo1.drop(demo1[demo1['Income'] == 'Prefer not to say'].index, inplace = True)
for i in range(0,len(demo1)-1):
    if demo1['Income'].loc[i].__contains__('100,000') or demo1['Income'].loc[i].__contains__('150,000') or demo1['Income'].loc[i].__contains__('200,000'):
        demo1.drop([i],inplace=True)
Monthly_ebike_propBy_Income(demo1)

In [ ]:
#returns trip purpose proportion of ebike trips by month excluding all NaN and 0 rows
import math
def ebike_month_purpose(trips):
    df = [rows for _, rows in trips.groupby(['start_local_dt_year','start_local_dt_month'])] #spilts dataframe into month-year
    count = trips.start_local_dt_month.value_counts().nunique() #count is the number of unique months in the dataset
    countOfPurposes = trips.purpose_confirm.value_counts().nunique()
    numOfPurposes={}
    user={}
    for i in range(len(df)):  #len(df) should be 17
        numOfPurposes['num'+str(i)]=df[i].purpose_confirm.value_counts().nunique() #number of unique users in each month
    for i in range(0, len(df)):
        user['month'+str(i)] = [rows for _, rows in df[i].groupby('purpose_confirm')] #f['u'+str(i)] is the month, the indexs are the user      
    timeline = pd.DataFrame({'December 2020':[],'January 2021':[],'February 2021':[],'March 2021':[],'April 2021':[],'May 2021':[],'June 2021':[],'July 2021':[],'August 2021':[],'September 2021':[],'October 2021':[],'November 2021':[],'December 2021':[],'January 2022':[],'February 2022':[],'March 2022':[],'April 2022':[],'May 2022':[]})
    for i in range(len(df)): #iterates through month  |   these for loops calculate percent of ebike usage per month
        for j in range(numOfPurposes['num'+str(i)]): #iterates through users per month
            #print(i,j)
            if user['month'+str(i)][j].mode_confirm.value_counts().__contains__('pilot_ebike'):
                ebike =user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            else:
                ebike = 0
            #ebike = user['month'+str(i)][j].mode_confirm.value_counts()['pilot_ebike']
            if sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True)) != 0:
                sumx = sum(user['month'+str(i)][j].mode_confirm.value_counts(dropna=True))
                percent = ebike/sumx
            #else:
            #    percent = 0
            #return percent
            #timeline = timeline.append()
            if user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2020:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'December 2020']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'January 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'February 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'March 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'April 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'May 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==6 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'June 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==7 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'July 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==8 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'August 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==9 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'September 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==10 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'October 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==11 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'November 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==12 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2021:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'December 2021']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==1 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'January 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==2 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'February 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==3 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'March 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==4 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'April 2022']=percent
            elif user['month'+str(i)][j].start_local_dt_month.iloc[0]==5 and user['month'+str(i)][j].start_local_dt_year.iloc[0]==2022:
                timeline.at[str(user['month'+str(i)][j].purpose_confirm.iloc[0]),'May 2022']=percent
    #return timeline
    #pd.set_option('display.max_rows', None)
    #timeline.drop([64]) 
    #timeline.drop([64], axis=0, inplace=True)
    #timeline.drop(['gas','move'],inplace=True)
    #return timeline.loc['entertainment'][0]==0
    #return timeline.loc['entertainment'][0]
    fig, ax = plt.subplots(figsize=(16,4))
    plt.title('Proportion of E-Bike Usage Over Time Based on Trip Purpose')
    plt.xlabel("Date")
    plt.ylabel("Proportion of e-bike trips")
    #timeline.reset_index(inplace=True)
  #  timeline1=timeline.drop([9])
    for i in timeline.index:
        count=0
        for j in range(17):
            if math.isnan(timeline.loc[i][j]) or timeline.loc[i][j]==0:
                count+=1
        if count == 17:
            timeline.drop([i],inplace=True)
    timeline.loc[len(timeline.index)]=[pd.to_datetime('2020-12'),pd.to_datetime('2021-01'),pd.to_datetime('2021-02'),pd.to_datetime('2021-03'),pd.to_datetime('2021-04'),pd.to_datetime('2021-05'),pd.to_datetime('2021-06'),pd.to_datetime('2021-07'),pd.to_datetime('2021-08'),pd.to_datetime('2021-09'),pd.to_datetime('2021-10'),pd.to_datetime('2021-11'),pd.to_datetime('2021-12'),pd.to_datetime('2022-01'),pd.to_datetime('2022-02'),pd.to_datetime('2022-03'),pd.to_datetime('2022-04'),pd.to_datetime('2022-05')]
    #return timeline
    #timeline=timeline.iloc[[0,1,3,5,7,8,15,16,21,65]]
    #return timeline
    for i in range(len(timeline)-1):
        sns.lineplot(ax=ax,x=timeline.loc[timeline.index[-1]],y=timeline.iloc[i],data=timeline,marker='o')
        plt.legend(bbox_to_anchor=(1, 1), loc='upper left', labels = timeline.index)
        plt.show

ebike_month_purpose(participant)

# Travel Patterns

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, participant)

In [ ]:
#Mode share for PARTICIPANT USERS ONLY
labels_mc = participant['Mode_confirm'].value_counts(dropna=True).keys().tolist()
values_mc = participant['Mode_confirm'].value_counts(dropna=True).tolist()
plot_title= "Number of trips for each mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_mode_confirm%s.png' % file_suffix
pie_chart_mode(plot_title,labels_mc,values_mc,file_name)
print(participant['Mode_confirm'].value_counts(dropna=True))

In [ ]:
labels_mc = participant.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True).keys().tolist()
values_mc = participant.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True).tolist()
plot_title= "Number of commute trips for each mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_commute_mode_confirm%s.png' % file_suffix
pie_chart_mode(plot_title,labels_mc,values_mc,file_name)
print(participant.query("Trip_purpose == 'Work'").Mode_confirm.value_counts(dropna=True))

In [ ]:
participant.loc[participant['Replaced_mode'] == 'Pilot ebike'].Mode_confirm.unique()

In [ ]:
labels_d10 = participant.loc[(participant['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True).keys().tolist()
values_d10 = participant.loc[(participant['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True).tolist()
plot_title="Mode confirmations for trips under 10 Miles\n%s" % quality_text
file_name ='ntrips_under10miles_mode_confirm%s.png' % file_suffix
pie_chart_mode(plot_title,labels_d10,values_d10,file_name)
print(participant.loc[(participant['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True))

In [ ]:
miles = participant.groupby('Mode_confirm').agg({'distance_miles': ['sum', 'count' , 'mean']})
miles.columns = ['Total (miles)', 'Count', 'Average (miles)']
miles = miles.reset_index()
miles =miles.sort_values(by=['Total (miles)'], ascending=False)

#data
miles_dict = dict(zip(miles['Mode_confirm'], miles['Total (miles)']))

labels_m = []
values_m = []

for x, y in miles_dict.items():
    labels_m.append(x)
    values_m.append(y)

plot_title="Miles for each mode (selected by users)\n%s" % quality_text
file_name ='miles_mode_confirm%s.png' % file_suffix
pie_chart_mode(plot_title,labels_m,values_m,file_name)
print(miles)

Energy Analysis

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook

%store -r df_EI
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
participant['Mode_confirm_fuel']= participant['Mode_confirm'].map(dic_fuel)
participant['Replaced_mode_fuel']= participant['Replaced_mode'].map(dic_fuel)

In [ ]:
participant = scaffolding.energy_intensity(participant, df_EI, 'distance','Replaced_mode', 'Mode_confirm')
participant = scaffolding.energy_impact_kWH(participant, 'distance_miles','Replaced_mode', 'Mode_confirm')

In [ ]:
data=participant.loc[(participant['distance_miles'] <= 40)].sort_values(by=['Energy_Impact(kWH)'], ascending=False)   
x='Energy_Impact(kWH)'
y='distance_miles'
legend ='Mode_confirm'
plot_title="Sketch of Energy Impact (kWH) by Travel Mode Selected\n%s" % quality_text
file_name ='sketch_distance_energy_impact%s.png' % file_suffix
distancevsenergy(data,x,y,legend,plot_title,file_name)

In [ ]:
#eirp : energy impact replaced_mode
eirc=participant.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eirc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eirc = eirc.reset_index()
eirc = eirc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
eirc['boolean'] = eirc['Sketch of Total Energy_Impact(kWH)'] > 0

#eimc : energy impact mode_confirm
eimc=participant.groupby('Mode_confirm').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eimc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eimc = eimc.reset_index()
eimc = eimc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)


subset1 = eirc [['Replaced_mode', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset1.rename(columns = {'Replaced_mode':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Replaced_Mode' }, inplace=True)

subset2 = eimc [['Mode_confirm', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset2.rename(columns = {'Mode_confirm':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Mode_Confirm' }, inplace=True)

df_plot = pd.merge(subset1, subset2, on="Transport Mode")
df = pd.melt(df_plot , id_vars=['Transport Mode'], value_vars=['Replaced_Mode','Mode_Confirm'], var_name='selection')
df.rename(columns = {'value':'Energy Impact (kWH)'}, inplace = True)

In [ ]:
#Sketch of Energy Impact by E-Bike Trips
data_eb = participant.query("Mode_confirm == 'Pilot ebike'")
# ebei : ebike energy impact
ebei=data_eb.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
ebei.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
ebei= ebei.reset_index()
ebei = ebei.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
ebei['boolean'] = ebei['Sketch of Total Energy_Impact(kWH)'] > 0
net_energy_saved = round(sum(ebei['Sketch of Total Energy_Impact(kWH)']), 2)

x = ebei['Sketch of Total Energy_Impact(kWH)']
y = ebei['Replaced_mode']
color =ebei['boolean']

plot_title="Sketch of Energy Impact of E-Bike trips\n Contribution by replaced mode towards a total of %s (kWH)\n %s" % (net_energy_saved, quality_text)
file_name ='sketch_energy_impact_ebike%s.png' % file_suffix
energy_impact(x,y,color,plot_title,file_name)

In [ ]:
expanded_ct = scaffolding.CO2_impact_lb(expanded_ct,'distance_miles','Replaced_mode', 'Mode_confirm')

In [ ]:
data_eb = participant.query("Mode_confirm == 'Pilot ebike'")

ebco2=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(lb)': ['sum', 'mean']},)
ebco2.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
ebco2 = ebco2.reset_index()
ebco2 = ebco2.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
ebco2['boolean'] = ebco2['total_lb_CO2_emissions'] > 0
net_CO2_emissions = round(sum(ebco2['total_lb_CO2_emissions']), 2)

x = ebco2['total_lb_CO2_emissions']
y = ebco2['Replaced_mode']
color = ebco2['boolean']

plot_title="Sketch of Total Pounds of CO2 Emissions of E-Bike trips\n Contribution by replaced mode towards a total of %s (lb CO2 Emissions )\n %s" % (net_CO2_emissions, quality_text)
file_name ='sketch_CO2impact_ebike%s.png' % file_suffix
CO2_impact(x,y,color,plot_title,file_name)